### Importamos las librerias necesarias para realizar nuestro ETL:

In [1]:
from pyspark.sql.functions import concat_ws, col, sum, lower, substring, to_date
from pyspark.sql import SparkSession

### Creacion de tabla `processed_files_reviews_yelp` para mantener el registro de los archivos ya procesados.

In [2]:
spark.sql("CREATE TABLE IF NOT EXISTS processed_files_reviews_yelp (file_name STRING) USING DELTA")

### Obtenemos los archivos sin procesar desde ADLS.

In [3]:
# Crear una instancia de SparkSession
spark = SparkSession.builder.getOrCreate()

# Especificar el nombre del contenedor y directorio
container_name = "datumtech"
directory_path = "/Yelp/reviews"

# Obtener la lista de carpetas
adls_files = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
    .listStatus(spark._jvm.org.apache.hadoop.fs.Path(f"abfss://{container_name}.blob.core.windows.net/{directory_path}"))

### Creamos un variable `new_files` que contiene los archivos que no estan procesados en ADLS.

In [4]:
processed_files = spark.sql("SELECT file_name FROM processed_files_reviews_yelp").toPandas()["file_name"].tolist()

new_files = [file.getPath().getName() for file in adls_files if file.getPath().getName() not in processed_files]

### Podemos ver que archivos ya estan procesados en la tabla `processed_files_metadata`

In [5]:
#spark.sql("SELECT * FROM processed_files_reviews_yelp").show()

### Podemos ver que archivos no estan procesados aun.

In [10]:
for file in new_files:
    print(file)

### Creamos la funcion `etl` que realiza todo el proceso y devuelve el dataframe en formato parquet a la tabla silver corrrespondiente a los datos ya procesados.

In [12]:

def etl(file):
    
    # Definimos las rutas:
    path_raw=f"abfss://datumtech@datumlake.dfs.core.windows.net/Yelp/reviews/{file}.json"
    path_bronze = f"abfss://datumtech@datumlake.dfs.core.windows.net/bronze/Yelpbronze/reviews-bronze/{file}-bronze"
    path_silver = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/Yelpsilver/reviews-silver/{file}-silver"

    # Cargamos el archivo desde ADLS. Nos quedamos solo con las columnas consideradas para el proyecto:
    df_raw = spark.read.format("json").load(path_raw).select('review_id', 'user_id', 'business_id', 'stars', 'text', 'date')
     
    # Guardamos el DataFrame df_raw en la tabla bronze correspondiente a los datos en crudo o poco procesados en Azure Data Lake con un formato parquet ideal para manejar altos volumenes de datos.
    df_raw.write.format("parquet").save(path_bronze)
        
    # Cargamos el archivo desde la tabla bronze en Azure Data Lake en un DataFrame.
    df_reviews = spark.read.format("parquet").load(path_bronze) 

    # Eliminar los duplicados
    df_reviews = df_reviews.dropDuplicates()

    # Eliminamos los registros donde haya nulos o vacios.
    df_reviews = df_reviews.na.drop()
    
    # Extraemos la parte de la fecha que nos interesa utilizando la función substring
    df_reviews = df_reviews.withColumn("date", substring(df_reviews.date, 1, 10))

    # Aplicamos la conversión a formato de fecha utilizando la función to_date
    df_reviews = df_reviews.withColumn("date", to_date(df_reviews.date, "yyyy-MM-dd"))

    # Guardamos el DataFrame df_metadata en la tabla silver correspondiente a los datos procesados en Azure Data Lake.
    return df_reviews.write.format("parquet").save(path_silver)
    

### Iteramos sobre cada archivo sin procesar.

In [13]:
for file in new_files:
    etl(file.rstrip(".json"))

### Agregamos a la tabla `processed_files_reviews_yelp` los archivos ya procesados.

In [14]:
new_files_df = spark.createDataFrame([(file,) for file in new_files], ["file_name"])
new_files_df.write.format("delta").mode("append").saveAsTable("processed_files_reviews_yelp")

### Podemos verificar que, efectivamente esten registrados los archivos ya procesados.

In [15]:
#spark.sql("SELECT * FROM processed_files_reviews_yelp").show()

#### Para comprobar que todo se haya ejecutado de manera correcta, podemos traer cualquier archivo de la tabla silver y hacer algunas verificaciones.

In [16]:
path = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/Yelpsilver/reviews-silver/review-silver"
df = spark.read.format("parquet").load(path)

In [18]:
df.show(5)

In [19]:
df.dtypes

In [20]:
df.count()

### Podemos verificar si hay nulos en algun archivo en la tabla silver.

In [21]:
# Funcion para el conteo de nulos del dataframe.
def null_counts (df):
    counts = df.select([sum(col(c).isNull().cast("integer")).alias(c) for c in df.columns])
    return counts.show()

In [22]:
# Vemos que columnas poseen nulos y en que cantidad.
nulls = null_counts(df)